In [1]:
##imports from libraries
import pandas as pd
import numpy as np
import time
import math
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import resource
from pathlib import Path


In [44]:
## Preprocessing of data
# Load data here:
path = Path(Path().absolute())
DataPath=str(path.parent)+'/Data/'

#taking the last element of data as y
X_=np.ndarray((0,16*327-1))
Y_=np.ndarray((0,1))

#2921
for i in range(2920):
    #print(i)
    f_name=DataPath+'ghg_data/ghg.gid.site'+format(i+2,'04d')+'.dat'
    #print(f_name)
    data = pd.read_csv(f_name, sep=" ",header=None)
    #print(data.shape)
    X = data.iloc[:,:]
    Y = data.iloc[15,-1]

    X = np.array(X).reshape(1,X.size)[:,0:-1]
    Y = np.array(Y).reshape(1,Y.size)
    X[np.isnan(X)] = 0
    Y[np.isnan(Y)] = 0
    
    X = X.astype(float)
    Y = Y.astype(float)
    
    X_=np.append(X_,X,axis=0)
    Y_=np.append(Y_,Y,axis=0)
    


X_=X_.T
Y_=Y_.T

print(X_.shape)
print(Y_.shape)

pd.DataFrame(X_).to_csv(DataPath+"data_x.csv",index=None)
pd.DataFrame(Y_).to_csv(DataPath+"data_y.csv",index=None)


(5231, 2920)
(1, 2920)


In [69]:
# Split train and test data here: (X_train, Y_train, X_test, Y_test)
X_train=X_[:,0:X_.shape[1]//2]
Y_train=Y_[0,0:Y_.shape[1]//2]

X_test=X_[:,X_.shape[1]//2:]
Y_test=Y_[0,Y_.shape[1]//2:]

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)




(5231, 1460)
(5231, 1460)
(1460,)
(1460,)


# Compute the gradient

The gradient of f(w) can be derived as follows:
\begin{align}
\nabla f= \frac{1}{N}\sum_{i\in[N]}\frac{-y_i}{e^{y_i w^T x_i}+1}x_i + 2\lambda w
\end{align}

In [105]:
## Logistic ridge regression with different optimizers
# cost function and gradient calculation

def cost(x,y,w,lambda_ = 0.01):
    D, N = x.shape
    value = 0
    for i in range(N):
        Z = -1 * y[i] * np.matmul(w.T , (x[:,i]).reshape(D,1))
        value += np.log(1+np.exp(Z))
    norm_w = np.linalg.norm(w)
    c = lambda_ * norm_w ** 2
    return value/N + c 

def function_gradient(X, Y, w, lambda_):
    D,N=X.shape
    value = np.zeros((D,1))
    for i in range(N):       
        Z = Y[i] * np.matmul(w.T , (X[:,i]).reshape(D,1))
        Q=(Y[i]/(1+np.exp(Z)))*(X[:,i]).reshape(D,1)        
        value += Q + 2*lambda_*w
        
    return value/N


[[3.30874759e-05]
 [3.30680220e-05]
 [2.65056001e-03]
 ...
 [5.47180046e+00]
 [5.21106825e+00]
 [4.94483099e+00]]


/home/sina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp


In [121]:
## Define solvers: GD, SGD, SVRG and SAG. 
# Setting the values here:

alpha = 0 # change the value
num_iters = 0 # change the value
lambda_ = 0 # change the value
epsilon = 0 # change the value

# ---------------------- Complete the blank definitions: --------------------------------------

def solver(x,y, w, alpha, num_iters , lambda_ , epsilon , optimizer = "GD",mem=False):
    if (optimizer == "GD") :
        for i in range(num_iters):
            # update the parameter w for GD here:
            g=function_gradient(x, y, w, lambda_)
            #print(np.linalg.norm(g))
            
            w=w-alpha*g
            if (i%10==0) and (mem):
                usage=resource.getrusage(resource.RUSAGE_SELF)
                print("mem for GD (mb):", (usage[2]*resource.getpagesize())/1000000.0)
            if (np.linalg.norm(g) <= epsilon):
                break
    elif (optimizer == "SGD"):
        for i in range(num_iters):
            print(i)
            # Complete SGD here:
                
    elif (optimizer == "SVRG"):
        T = 100
        K = math.floor(num_iters/T)
        Z = np.matmul(x,np.diagflat(y))
        N = x.shape[1]
        for k in range(K):
            wz = np.matmul(w.T , Z)
            diag = np.diagflat(1/(1+np.exp(-1*wz))-np.ones((1,N)))
            Ga_ = np.matmul(Z , diag)
            ga_ = (1/N) * np.matmul(Ga_ , np.ones((N,1)))
            for t in range(T):
                # Complete SVRG here:
                print(t)
            
    elif (optimizer == "SAG"):
        # Complete SAG here:
        print(x)
            
    return w

In [123]:
## Solving the optimization problem:

#y = np.array(Y_train.iloc[0:6000])
#x = np.array(X_train.iloc[0:6000,:])
D,N = X_train.shape
w = np.random.rand(D,1)*0.01  # Initialization of w

#-------------------- GD Solver -----------------------
start=time.time()
num_iters = 50
alpha=0.5
lambda_=0.01
epsilon=0.0001
gde = solver(X_train,Y_train,w,alpha,num_iters,lambda_,epsilon,optimizer="GD",mem=False) # complete the command 
end = time.time()
print("Weights of GD after convergence: \n",gde)
cost_value = cost(X_test,Y_test,gde,lambda_)
print("Cost of GD after convergence: ",cost_value)
print("Training time for GD: ", end-start , ' sconds')

asd

#-------------------- SGD Solver -----------------------
# complete here :

#-------------------- SVRG Solver -----------------------
# complete here :

#-------------------- SAG Solver -----------------------
# complete here :

/home/sina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp


Weights of GD after convergence: 
 [[0.0045957 ]
 [0.00372056]
 [0.00545639]
 ...
 [0.00499467]
 [0.00065194]
 [0.00520025]]
Cost of GD after convergence:  [[0.00058977]]
Training time for GD:  3.592517614364624  sconds


NameError: name 'asd' is not defined

In [ ]:
## Executing the iterations and plot the cost function here:

ti= np.zeros((50,4))
cost_= np.zeros((50,4))
for i in range(50):
    print("......",i,".......")
    #--------------GD-------------------
    start = time.time()
    gde = solver(x.T,y,w,num_iters=i)
    end = time.time()

    cost_[i,0] = cost(x.T,y,gde)

    ti[i,0] = end-start

    #---------------SGD------------------
    #Complete for SGD solver here :
    
    #---------------SVRG----------------
    #Complete for SVRG solver here :
    
    #---------------SAG------------------
    #Complete for SAG solver here :
    
    #------------------------------------
    
    ## Pl the results:
    

l0 = plt.plot(cost_[:,0],color="red")
# complete other plots here: 


plt.xlabel("Number of Iteration")
plt.ylabel("Cost")
plt.legend(['GD', 'SGD', 'SVRG', 'SAG'])
plt.show()

l0 = plt.plot(ti[:,0],color="red")
# complete other plots here:

plt.xlabel("Number of Iteration")
plt.ylabel("Time (sec)")
plt.legend(['GD', 'SGD', 'SVRG', 'SAG'])
plt.show()
    
    
    

In [ ]:
## Tunning the hyper-paramter here:

In [ ]:
## Comparing different optimizers here: 